- 대리점의 매출 이력, 고객 정보 데이터 이용 → 분석과 예측을 하기 위한 데이터 가공 학습
- 대리점 데이터 : 시스템에 의해 관리되는 쇼핑몰과 달리 사람의 손을 타는 편 → 입력 실수, 데이터 누락 등 오류 포함
- '지저분한' 데이터를 다루는 연습이 필요

### 데이터
- uriage.csv : 매출 이력(201901 ~ 201907)
- kokyaku_daicho.xlsx : 고객 정보

#### 테크닉 011 : 데이터를 읽어 들이자

In [1]:
# 매출 이력 데이터
import pandas as pd
uriage_data = pd.read_csv('/content/drive/My Drive/pyda100/pyda100-master/2장/uriage.csv')
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [2]:
# 고객 정보 데이터(엑셀)
kokyaku_data = pd.read_excel('/content/drive/My Drive/pyda100/pyda100-master/2장/kokyaku_daicho.xlsx')
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


- 데이터들에 결측치가 있고, 공백, 표기 방법 차이 등 데이터 오류가 발생
- 정합성을 갖추기 위해, 데이터의 속성, 의미를 이해하는 것이 중요
- 데이터의 오류를 파악하는 것부터 시작

### 테크닉 012 : 데이터의 오류를 살펴보자

In [3]:
uriage_data['item_name'].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

- 공백 포함, 대소문자 표기 방법 등의 오류 발생

In [4]:
uriage_data['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

- 결측치 NaN 확인

### 테크닉 013 : 데이터에 오류가 있는 상태로 집계해 보자

데이터의 오류가 집계에 어느 정도 영향을 미치는지 확인해보는 과정

In [5]:
# 매출 이력 → 월 매출 합계 집계
# 날짜 → 연월 칼럼 생성
uriage_data['purchase_date'] = pd.to_datetime(uriage_data['purchase_date'])
uriage_data['purchase_month'] = uriage_data['purchase_date'].dt.strftime('%Y%m')
# pivot table 생성(fill_value = 0 : 결측치 대체)
res = uriage_data.pivot_table(index = 'purchase_month', columns = 'item_name', aggfunc = 'size', fill_value = 0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,2,0,1,0,0,0,0,0,...,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,...,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,...,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0


- 상품S, 상품s와 같이 동일한 상품이 다른 상품으로 집계됨
- 상품의 개수가 26개에서 99개로 늘어남

In [6]:
# item_price 집계
res = uriage_data.pivot_table(index = 'purchase_month', columns = 'item_name', values = 'item_price', aggfunc = 'sum', fill_value = 0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,1500,1600,2000,0,0,4800,0,1900,0,0,0,0,0,...,6000,6500,7800,7000,10400,13500,16000,16500,19200,19500,21000,27000,20800,25500,36000,28500,28000,12600,41800,27600,24000,20000,0,0,300,0,0,0,0,0,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,0,0,0,0,0,0,0,0,400,0,0,0,0,900,0,0,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7600,5500,6600,7000,12800,8100,11000,14300,13200,16900,23800,25500,30400,30600,32400,32300,40000,37800,39600,43700,31200,20000,2600,0,0,0,0,0,0,1000,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6000,4000,11400,7700,14400,10800,13000,14300,21600,26000,16800,15000,32000,34000,27000,26600,22000,46200,35200,34500,38400,35000,0,0,0,400,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,2000,2200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1500,0,0,0,0,0,7500,0,0,0,0,1200,0,...,7200,8000,7800,9100,7200,14400,13000,12100,15600,14300,22400,18000,22400,28900,30600,28500,28000,16800,19800,20700,38400,32500,0,0,0,0,500,700,0,0,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,0,0,0,100,0,1900,100,0,0,0,0,0,0,0,1100,1300,0,0,0,0,0,2200,0,0,0,0,0,0,600,0,...,5200,9000,9000,14000,12000,13500,8000,15400,12000,20800,23800,24000,28800,15300,32400,20900,26000,33600,8800,36800,31200,45000,0,0,300,0,0,0,0,0,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,0,0,0,0,0,0,0,300,0,0,0,700,0,0,0,0,0,1500,0,0,2100,0,0,0,0,100,0,0,0,800,...,7200,6500,9600,7700,8800,12600,20000,15400,14400,19500,15400,19500,24000,23800,27000,34200,22000,29400,33000,25300,24000,30000,0,0,0,0,0,0,900,0,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1700,0,0,0,0,500,0,0,800,0,0,0,0,1500,0,0,0,2200,0,0,0,0,0,500,0,0,...,6000,4500,9600,11900,12800,13500,17000,9900,20400,13000,18200,25500,38400,18700,25200,38000,34000,23100,52800,32200,38400,27500,0,0,0,0,0,0,900,0,0,0,1500,0,1800,0,0,0,0,0


- 데이터 오류를 처리하지 않으면 의미 없는 결과가 발생함

### 테크닉 014 : 상품명 오류를 수정하자

- 수정 후의 결과가 올바른지 아닌지를 판정하기 위해 현재 상태를 파악하는 것이 중요

In [7]:
# 상품명의 유니크한 갯수 확인
len(pd.unique(uriage_data.item_name))

99

In [8]:
# 'item_name' 오류 수정
uriage_data['item_name'] = uriage_data['item_name'].str.upper() # 모든 알파벳을 대문자로
uriage_data['item_name'] = uriage_data['item_name'].str.replace('  ', '') # 공백 두 칸 제거
uriage_data['item_name'] = uriage_data['item_name'].str.replace(' ', '') # 공백 한 칸 제거
uriage_data.sort_values(by = ['item_name'], ascending = True) # item_name 순으로 정렬

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [9]:
# 결과 검증
print(len(pd.unique(uriage_data['item_name'])))
print(pd.unique(uriage_data['item_name']))

26
['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']


### 테크닉 015 : 금액의 결측치를 수정하자

In [10]:
# 결측치 확인
uriage_data.isnull().any(axis = 0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

- item_price  칼럼에서 결측치 확인
- 단가 변동이 없다는 전제 조건 → 같은 상품의 단가 이용, 결측치 보간

In [11]:
# 결측치가 존재하는 곳 확인, flg_is_null 변수에 저장
flg_is_null = uriage_data['item_price'].isnull()

# 결측치가 존재하는 상품명 추출, 같은 상품의 다른 데이터에서 금액을 가져와 저장
for trg in list(uriage_data.loc[flg_is_null, 'item_name'].unique()):
  price = uriage_data.loc[(~flg_is_null) & (uriage_data['item_name'] == trg), 'item_price'].max()
  uriage_data['item_price'].loc[(flg_is_null) & (uriage_data['item_name'] == trg)] = price

uriage_data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [12]:
# 검증 진행
uriage_data.isnull().any(axis = 0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [13]:
# 금액 수정 여부 확인
for trg in list(uriage_data['item_name'].sort_values().unique()):
  print(trg + '의 최고가 : ' + str(uriage_data.loc[uriage_data['item_name'] == trg]['item_price'].max()) + ', ' 
        + trg + '의 최저가 : ' +  str(uriage_data.loc[uriage_data['item_name'] == trg]['item_price'].min(skipna = False)))

상품A의 최고가 : 100.0, 상품A의 최저가 : 100.0
상품B의 최고가 : 200.0, 상품B의 최저가 : 200.0
상품C의 최고가 : 300.0, 상품C의 최저가 : 300.0
상품D의 최고가 : 400.0, 상품D의 최저가 : 400.0
상품E의 최고가 : 500.0, 상품E의 최저가 : 500.0
상품F의 최고가 : 600.0, 상품F의 최저가 : 600.0
상품G의 최고가 : 700.0, 상품G의 최저가 : 700.0
상품H의 최고가 : 800.0, 상품H의 최저가 : 800.0
상품I의 최고가 : 900.0, 상품I의 최저가 : 900.0
상품J의 최고가 : 1000.0, 상품J의 최저가 : 1000.0
상품K의 최고가 : 1100.0, 상품K의 최저가 : 1100.0
상품L의 최고가 : 1200.0, 상품L의 최저가 : 1200.0
상품M의 최고가 : 1300.0, 상품M의 최저가 : 1300.0
상품N의 최고가 : 1400.0, 상품N의 최저가 : 1400.0
상품O의 최고가 : 1500.0, 상품O의 최저가 : 1500.0
상품P의 최고가 : 1600.0, 상품P의 최저가 : 1600.0
상품Q의 최고가 : 1700.0, 상품Q의 최저가 : 1700.0
상품R의 최고가 : 1800.0, 상품R의 최저가 : 1800.0
상품S의 최고가 : 1900.0, 상품S의 최저가 : 1900.0
상품T의 최고가 : 2000.0, 상품T의 최저가 : 2000.0
상품U의 최고가 : 2100.0, 상품U의 최저가 : 2100.0
상품V의 최고가 : 2200.0, 상품V의 최저가 : 2200.0
상품W의 최고가 : 2300.0, 상품W의 최저가 : 2300.0
상품X의 최고가 : 2400.0, 상품X의 최저가 : 2400.0
상품Y의 최고가 : 2500.0, 상품Y의 최저가 : 2500.0
상품Z의 최고가 : 2600.0, 상품Z의 최저가 : 2600.0


- 최고가 = 최저가 : 모든 상품의 금액이 정상적으로 수정됨
- min(skipna = False) : NaN 무시 여부 설정. False → 최솟값이 NaN으로 표시됨

### 테크닉 016 : 고객 이름의 오류를 수정하자

In [14]:
# 데이터 확인
kokyaku_data['고객이름'].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [15]:
uriage_data['customer_name'].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

- 고객 관리 데이터의 경우 공백이 혼재되어 있음 → 결합 시 정상적인 결합이 되지 않음
- 고객 이름을 반드시 수정해야 함

In [16]:
# 고객 이름 공백 제거
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace('  ', '')
kokyaku_data['고객이름'] = kokyaku_data['고객이름'].str.replace(' ', '')
kokyaku_data['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

### 테크닉 017 : 날짜 오류를 수정하자

- 엑셀 데이터 : 서식이 다른 데이터가 섞여 있을 수 있는 문제점 발생

In [17]:
flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit() # 등록일이 숫자인지 아닌지의 여부를 isdigit()이용하여 판정, flg_is_serial에 저장
flg_is_serial.sum() # 숫자 데이터 개수 확인

22

In [18]:
# 숫자로 저장된 등록일 데이터를 날짜로 변환
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '등록일'].astype('float'), unit = 'D') + pd.to_datetime('1900/01/01')
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [19]:
# 서식 통일
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '등록일'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [20]:
kokyaku_data['등록일'] = pd.concat([fromSerial, fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [22]:
# 등록월 추출
kokyaku_data['등록연월'] = kokyaku_data['등록일'].dt.strftime('%Y%m')
rslt = kokyaku_data.groupby('등록연월').count()['고객이름']
print(rslt)
print(len(kokyaku_data))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


In [23]:
# 등록일 칼럼에 숫자 데이터가 남아 있는지 확인
flg_is_serial = kokyaku_data['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

0

### 테크닉 018 : 고객 이름을 키로 두 개의 데이터를 결합(조인)하자

- 매출 이력 + 고객 정보 결합
- 두 개의 데이터의 서로 다른 열을 지정해서 결합

In [25]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on = 'customer_name', right_on = '고객이름', how = 'left')
join_data = join_data.drop('customer_name', axis = 1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,201907,김승주,E시,2018-07-15,201807


- left_on, right_on 으로 결합할 데이터 지정
- left_on의 인수 : uriage_data의 customer_name
- right_on의 인수 : kokyaku_data의 고객 이름
- how = left : uriage_data를 기준으로 kokyaku_data 결합

### 테크닉 019 : 정제한 데이터를 덤프하자

- 가공 및 정제가 끝난 데이터를 파일로 출력(덤프)
- 칼럼의 위치를 재배치한 후 파일로 저장

In [27]:
# 칼럼 위치 재배치
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name', 'item_price', '고객이름', '지역', '등록일']]
dump_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15


In [28]:
# csv파일로 저장
dump_data.to_csv('/content/drive/My Drive/pyda100/pyda100-master/2장/dump_data.csv', index = False)

### 테크닉 020 : 데이터를 집계하자

In [29]:
# 덤프 파일 불러오기
import_data = pd.read_csv('/content/drive/My Drive/pyda100/pyda100-master/2장/dump_data.csv')
import_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07 00:00:00
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01 00:00:00
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31 00:00:00
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15 00:00:00
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15 00:00:00


In [30]:
# purchase_month를 세로축 → 상품별 집계
byItem = import_data.pivot_table(index = 'purchase_month', columns = 'item_name', aggfunc = 'size', fill_value = 0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,20,19,19,16,24,16,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,16,11,15,20,19,19,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,14,20,22,13,11,21,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,14,15,11,20,15,15,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,18,13,18,19,18,20,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,15,15,17,16,15,18,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,12,17,11,15,22,26,15,19,23,21,13,28,16,18,12,0


In [32]:
# purchase_month를 세로축 → 매출 금액 집계
byPrice = import_data.pivot_table(index = 'purchase_month', columns = 'item_name', values = 'item_price', aggfunc = 'sum', fill_value = 0)
byPrice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,22000,22800,24700,22400,36000,25600,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,17600,13200,19500,28000,28500,30400,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,15400,24000,28600,18200,16500,33600,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,15400,18000,14300,28000,22500,24000,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,19800,15600,23400,26600,27000,32000,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,16500,18000,22100,22400,22500,28800,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,13200,20400,14300,21000,33000,41600,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [33]:
# purchase_month를 세로축 → 고객 집계
byCustomer = import_data.pivot_table(index = 'purchase_month', columns = '고객이름', aggfunc = 'size', fill_value = 0)
byCustomer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,김도경,김도영,김도원,김도윤,김도율,김도하,김도현,김도훈,김동건,김동우,김동욱,김동하,김동현,김라온,김민건,김민규,김민기,김민서,김민석,김민수,김민우,김민재,김민준,김민찬,김민혁,김민호,김범준,김상민,김상현,김서윤,...,김지혁,김지호,김지환,김지후,김지훈,김진우,김찬영,김찬우,김태경,김태민,김태양,김태영,김태우,김태윤,김태인,김태준,김태현,김태훈,김하람,김하랑,김하민,김하윤,김하율,김하준,김하진,김한결,김현민,김현서,김현석,김현성,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,4,0,0,2,2,2,0,3,2,2,2,1,3,3,3,1,3,1,4,2,0,3,3,0,4,4,5,1,2,2,...,3,5,2,2,2,1,6,5,1,2,1,2,3,4,0,0,0,1,1,2,3,3,2,1,7,1,0,1,3,2,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,1,6,4,4,1,2,4,3,1,2,0,1,1,3,1,1,3,2,2,3,3,3,9,0,5,2,2,0,1,2,...,3,0,1,1,1,1,1,3,4,1,1,0,1,2,2,0,0,1,2,3,3,2,1,0,2,0,4,4,5,3,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,3,0,3,3,0,2,4,2,5,1,1,0,3,1,2,1,5,5,1,0,1,5,1,1,1,1,3,3,0,6,...,4,4,2,1,1,3,0,3,2,2,3,4,3,2,4,2,4,3,1,2,2,1,2,1,1,4,0,5,0,2,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,1,3,1,3,1,4,2,2,1,4,4,1,1,3,2,2,4,2,0,1,2,1,0,2,5,4,2,3,2,2,...,1,4,3,1,0,4,1,1,1,5,4,1,0,0,2,1,3,5,0,5,3,4,4,4,1,0,1,4,4,3,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,3,2,3,2,2,1,1,1,2,2,0,2,4,4,0,1,2,2,2,1,3,1,3,5,2,1,2,1,2,2,...,3,2,6,3,4,0,0,2,2,4,2,5,1,0,3,1,1,1,1,5,4,4,2,1,1,2,2,2,0,2,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,2,2,5,4,2,1,1,2,1,3,1,2,2,3,3,4,3,3,6,2,1,2,1,2,0,2,1,3,4,4,...,0,1,3,1,1,2,2,1,4,4,1,3,2,3,2,3,1,2,2,0,2,3,2,3,0,1,1,1,1,4,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,4,2,1,1,3,2,3,1,1,3,5,2,1,2,2,1,1,2,2,1,5,0,3,4,2,2,5,4,4,2,...,2,0,1,2,5,2,2,4,6,1,1,5,2,2,4,1,2,2,2,2,2,2,2,4,1,0,1,3,2,4,2,1,0,4,0,2,6,1,2,3


In [34]:
# purchase_month를 세로축 → 지역 집계
byRegion = import_data.pivot_table(index = 'purchase_month', columns = '지역', aggfunc = 'size', fill_value = 0)
byRegion

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [36]:
# 집계 기간 중 구매 이력이 없는 사용자 확인
away_data = pd.merge(uriage_data, kokyaku_data, left_on = 'customer_name', right_on = '고객이름', how = 'right')
away_data[away_data['purchase_date'].isnull()][['고객이름', '등록일']]

,고객이름,등록일
2999,김서우,2019-04-23


### 정리
1.피봇 테이블 생성

table = data.pivot_table(index = '세로축', columns = '가로축', values = '넣을 값', aggfunc = '사용할 함수', fill_value = 0)

* values에 [A, B] 와 같이 두 개의 칼럼을 넣으면 좌우 두개의 피봇 테이블이 생성됨

2.결측치, 데이터타입, 데이터표기 방식 등 오류 확인 및 제거
